#### Problem statement:
Tomato plants are susceptible to various diseases that can significantly impact yield and quality. Early and accurate detection of these diseases is crucial for effective management and control. Traditional methods of disease detection are time-consuming and require expert knowledge. Our objective is to develop a Convolutional Neural Network (CNN) model to classify tomato plant diseases from images with high accuracy.

#### Importing necessary modules:

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import pickle

#### Loading the dataset:

Dataset credits: https://www.kaggle.com/datasets/arjuntejaswi/plant-village 

In [2]:
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 25
CHANNELS = 3

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "tomato_disease_classes",
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True
)

Found 16011 files belonging to 10 classes.


#### Training, testing and validation split:

In [4]:
n_batches = len(dataset)
train_size = int(0.8*n_batches)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_size = int(0.1*train_size)
val_dataset = train_dataset.take(val_size)
train_dataset = train_dataset.skip(val_size)

#### Caching, shuffling and prefetching the dataset: 

In [5]:
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

#### Model definition, compilation and training:

In [6]:
#Layer for normalization:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(256, 256),
  layers.Rescaling(1./255),
])

In [7]:
#Layer for data augmentation:
augmentation = tf.keras.Sequential([
    layers.RandomFlip(mode="horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

In [8]:
#Defining the model architecture:
input_shape = (BATCH_SIZE, 256, 256, CHANNELS)
n_classes = 10
model = models.Sequential([
    resize_and_rescale,
    augmentation,
    layers.Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(n_classes, activation="softmax")
])
model.build(input_shape=input_shape)

c:\Users\singh\anaconda3\envs\CBTC\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (32, 254, 254, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (32, 125, 125, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (32, 62, 62, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (32, 60, 60, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (32, 30, 30, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 28, 28, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (32, 14, 14, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (32, 12, 12, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (32, 6, 6, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 2304)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 64)               │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 10)               │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,346 (1.06 MB)

 Trainable params: 278,346 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [11]:
history = model.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    validation_data=val_dataset,
    verbose=1,
    epochs=EPOCHS,
)

Epoch 1/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 386s 994ms/step - accuracy: 0.2913 - loss: 1.9489 - val_accuracy: 0.5094 - val_loss: 1.3082
Epoch 2/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 323s 897ms/step - accuracy: 0.5895 - loss: 1.1578 - val_accuracy: 0.6797 - val_loss: 0.9687
Epoch 3/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 342s 950ms/step - accuracy: 0.7446 - loss: 0.6959 - val_accuracy: 0.6484 - val_loss: 1.3299
Epoch 4/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 324s 899ms/step - accuracy: 0.7983 - loss: 0.5657 - val_accuracy: 0.7555 - val_loss: 0.7521
Epoch 5/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 323s 896ms/step - accuracy: 0.8363 - loss: 0.4604 - val_accuracy: 0.6711 - val_loss: 1.1329
Epoch 6/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 325s 902ms/step - accuracy: 0.8590 - loss: 0.3978 - val_accuracy: 0.8273 - val_loss: 0.4722
Epoch 7/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 321s 893ms/step - accuracy: 0.8774 - loss: 0.3406 - val_accuracy: 0.8867 - val_loss: 0.3339
Epoch 8/25
360/360 ━━━━━━━━━━━━━━━━━━━━ 322s 894ms/step - accuracy: 0.8848 -

#### Model evaluation:

In [12]:
scores = model.evaluate(test_dataset)

101/101 ━━━━━━━━━━━━━━━━━━━━ 38s 218ms/step - accuracy: 0.9367 - loss: 0.1910


#### Saving the model:

In [13]:
model.save("model.keras")